In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('axes', unicode_minus=False)
%matplotlib inline

np.random.seed(123)

In [ ]:
train = pd.read_csv('data/job_train.csv')
test = pd.read_csv('data/job_test.csv')
df = pd.concat([train,test])
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14668 entries, 0 to 14667
Columns: 109 entries, STU_ID to X1STU30OCC2
dtypes: float64(36), int64(73)
memory usage: 12.2 MB


In [ ]:
df.X1STU30OCC2.nunique()

21

In [ ]:
df.X1STU30OCC2.value_counts()

29.0    3479
27.0    2088
19.0     844
17.0     828
23.0     657
25.0     616
33.0     478
55.0     461
11.0     369
39.0     286
15.0     280
49.0     254
35.0     232
13.0     217
21.0     165
47.0     134
53.0     114
41.0      88
51.0      69
31.0      43
45.0      38
Name: X1STU30OCC2, dtype: int64

In [ ]:
df = df.where(~df.isin([-9,-8,-7]), np.nan)


In [ ]:
null_check = df.isnull().sum()

In [ ]:
sum_df = df.describe().T.iloc[:,3:]
sum_df['null_check'] = null_check
sum_df.to_csv('data/sum_df.csv')

In [ ]:
sum_df 

,min,25%,50%,75%,max,null_check
STU_ID,10001.00,16317.75,22661.50,28909.75,35205.00,0
X1SEX,1.00,1.00,2.00,2.00,2.00,0
X1RACE,1.00,5.00,8.00,8.00,8.00,0
X1HISPANIC,0.00,0.00,0.00,0.00,1.00,0
X1WHITE,0.00,0.00,1.00,1.00,1.00,118
...,...,...,...,...,...,...
X1COUPERCOU,-6.43,-0.69,0.26,1.02,1.15,1574
X1COUPERPRI,-5.51,-0.68,0.20,1.10,1.10,1912
X1AQSTAT,1.00,1.00,1.00,1.00,8.00,0
X1AQDATE,200910.00,200911.00,201001.00,201001.00,201001.00,746


In [ ]:
##nullcheck 어떻게 처리할지 정해야함

nullp = pd.read_csv('data/null_pre.csv')
nullp = nullp.dropna().reset_index(drop=True)
nullp = nullp.set_index('variables')['null_pre']
nullp

variables
STU_ID         0.00
X1SEX          0.00
X1RACE         0.00
X1HISPANIC     0.00
X1WHITE        0.00
               ... 
X1COUPERTEA    0.03
X1COUPERCOU    0.26
X1COUPERPRI    0.20
X1AQSTAT       0.00
X1STU30OCC2    0.00
Name: null_pre, Length: 103, dtype: float64

In [ ]:
#csv에 작성한 null값을 읽어서 채움
df = df[nullp.index]
df = df.fillna(nullp)

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df['birthyear'] = (df['X1STDOB']/100).astype('int')
df['birthyear'] = (df.birthyear % 1990).astype('int')
df['birthmonth'] = (df['X1STDOB']%100).astype('int')

In [ ]:
test = df[df.X1STU30OCC2==0].reset_index(drop=True)
test = test.drop(columns = ['X1STU30OCC2'])
train = df[df.X1STU30OCC2!=0].reset_index(drop=True)

In [ ]:
idx2job = train['X1STU30OCC2'].value_counts().reset_index().astype('int')
idx2job = idx2job[['index']]
idx2job.columns = ['job']
idx2job.to_csv('data/idx2job.csv')
idx2job = idx2job['job'].to_dict()
job2idx = {v:i for i,v in idx2job.items()}

In [ ]:
train['X1STU30OCC2'] = train['X1STU30OCC2'].replace(job2idx).astype('int')

In [ ]:
train.to_csv('data/train_ver02.csv', index=False)
test.to_csv('data/test_ver02.csv', index=False)

In [ ]:
train.shape

(11740, 105)

In [ ]:
test.shape

(2928, 104)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv('data/train_ver02.csv', index_col='STU_ID')
test = pd.read_csv('data/test_ver02.csv', index_col='STU_ID')
idx2job = pd.read_csv('data/idx2job.csv', index_col=0)
idx2job = idx2job['job'].to_dict()
train.head()

,X1SEX,X1RACE,X1HISPANIC,X1WHITE,X1BLACK,X1DUALLANG,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,...,X1LOCALE,X1REGION,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,X1AQSTAT,X1STU30OCC2,birthyear,birthmonth
STU_ID,,,,,,,,,,,,,,,,,,,,,
10002,2,8,0,1.0,0.0,1.0,199511.0,-0.2956,0.2696,35.8045,...,4,1,0.32,-0.80,1.15,-1.61,1,0,5,11
10003,2,3,0,0.0,1.0,1.0,199506.0,1.2997,0.2555,56.0477,...,2,4,-0.61,0.81,-0.15,0.00,1,5,5,6
10004,2,8,0,1.0,0.0,1.0,199505.0,-0.1427,0.2649,38.0888,...,2,3,-2.14,0.03,0.26,0.20,1,0,5,5
10005,1,8,0,1.0,0.0,1.0,199505.0,1.1405,0.2312,54.1762,...,1,3,-0.65,-0.25,0.14,0.05,1,10,5,5
10006,2,8,0,1.0,0.0,1.0,199504.0,0.7106,0.2567,49.0531,...,4,3,-0.31,0.51,-0.90,0.69,1,0,5,4


In [ ]:
y = train.X1STU30OCC2
x = train.drop(columns='X1STU30OCC2')

In [ ]:
forest = RandomForestClassifier(n_estimators=300, n_jobs=-1)
forest.fit(x,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [ ]:
# 0.37403
pred_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : forest.predict(test)})
pred_base['X1STU30OCC2'] = pred_base['X1STU30OCC2'].replace(idx2job)
pred_base.to_csv('jobrec_baseline.csv', index=False, encoding='utf-8-sig')

In [ ]:
imp = forest.feature_importances_
idx = np.argsort(imp)[::-1]
labels = x.columns
pd.DataFrame({'columns':x.columns, 'imp':forest.feature_importances_}).sort_values(by='imp', ascending=False)

,columns,imp
66,X1STUEDEXPCT,0.023531
0,X1SEX,0.022778
8,X1TXMSEM,0.022386
54,X1SES_U,0.020889
65,X1SCHOOLENG,0.020816
...,...,...
71,X1PQSTAT,0.001097
17,X1MACC,0.000923
69,X1TESTSTAT,0.000887
70,X1SQSTAT,0.000664


In [ ]:
x_filter = x[labels[imp>0.01]] #threshold를 줘서 피쳐 줄였음
x_filter.shape
test_filter = test[x_filter.columns]

In [ ]:
forest_small = RandomForestClassifier(n_estimators=300, n_jobs=-1)
forest_small.fit(x_filter,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [ ]:
#0.37959
g_small = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : forest_small.predict(test_filter)})
g_small['X1STU30OCC2'] = g_small['X1STU30OCC2'].replace(idx2job)
g_small.to_csv('jobrec_small.csv', index=False, encoding='utf-8-sig')

In [ ]:
train_new = x_filter
train_new['X1STU30OCC2'] = y.values
train_new.reset_index().to_csv('data/train_ver03.csv',index=False, encoding='utf-8-sig')
test_filter.reset_index().to_csv('data/test_ver03.csv',index=False, encoding='utf-8-sig')

/var/folders/vt/76d2nkdd2msb93lkngc6yv1r0000gn/T/ipykernel_3221/692845606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_new['X1STU30OCC2'] = y.values


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(123)

In [2]:
df = pd.read_csv('data/train_ver03.csv', index_col='STU_ID')
df.head()

,X1SEX,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,X1TXMPROF4,...,X1TSEFF,X1TSEXP,X1TSPRINC,X1TSRESP,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,birthmonth,X1STU30OCC2
STU_ID,,,,,,,,,,,,,,,,,,,,,
10002,2,199511.0,-0.2956,0.2696,35.8045,47.6821,0.9714,0.5204,0.1768,0.0582,...,-0.86,0.54,-0.13,-0.13,0.32,-0.80,1.15,-1.61,11,0
10003,2,199506.0,1.2997,0.2555,56.0477,64.2431,1.0000,0.9914,0.9838,0.6606,...,0.17,0.16,-0.13,-0.13,-0.61,0.81,-0.15,0.00,6,5
10004,2,199505.0,-0.1427,0.2649,38.0888,49.2690,0.9843,0.6416,0.2459,0.0662,...,0.17,0.16,-0.13,-0.13,-2.14,0.03,0.26,0.20,5,0
10005,1,199505.0,1.1405,0.2312,54.1762,62.5897,0.9999,0.9863,0.9686,0.5385,...,-0.25,1.37,-0.13,-0.30,-0.65,-0.25,0.14,0.05,5,10
10006,2,199504.0,0.7106,0.2567,49.0531,58.1268,0.9995,0.9539,0.8469,0.2463,...,-0.86,-1.12,-0.32,-0.13,-0.31,0.51,-0.90,0.69,4,0


In [3]:
y = df.X1STU30OCC2
x = df.drop(columns='X1STU30OCC2')
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)

In [4]:
y_train.value_counts(normalize=True)

0     0.296309
1     0.177853
2     0.071891
3     0.070528
4     0.055991
5     0.052470
6     0.040772
7     0.039296
8     0.031459
9     0.024304
10    0.023850
11    0.021579
12    0.019761
13    0.018512
14    0.014083
15    0.011357
16    0.009654
17    0.007496
18    0.005906
19    0.003634
20    0.003294
Name: X1STU30OCC2, dtype: float64

In [5]:
y_test.value_counts(normalize=True)

0     0.296422
1     0.177853
2     0.071891
3     0.070528
4     0.055877
5     0.052470
6     0.040545
7     0.039182
8     0.031346
9     0.024532
10    0.023850
11    0.021806
12    0.019761
13    0.018399
14    0.013969
15    0.011584
16    0.009881
17    0.007496
18    0.005792
19    0.003748
20    0.003066
Name: X1STU30OCC2, dtype: float64

In [6]:
import joblib
with open('data/my_val.pkl', 'wb') as f:
    joblib.dump({'x':x_train,
                 'y':y_train,
                 'val_x':x_test,
                 'val_y':y_test},f)

In [ ]:
import pandas as pd
import numpy as np
import joblib

In [ ]:
with open('data/my_val.pkl', 'rb') as f:
    data = joblib.load(f)
    print(data.keys())
locals().update(data)
del data
np.random.seed(123)

dict_keys(['x', 'y', 'val_x', 'val_y'])


In [ ]:
test = pd.read_csv('data/test_ver03.csv', index_col='STU_ID')
idx2job = pd.read_csv('data/idx2job.csv', index_col=0)
idx2job = idx2job['job'].to_dict()
test.head()

,X1SEX,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,X1TXMPROF4,...,X1TSCOMM,X1TSEFF,X1TSEXP,X1TSPRINC,X1TSRESP,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,birthmonth
STU_ID,,,,,,,,,,,,,,,,,,,,,
12424,1,199509.0,-0.1663,0.2511,37.7431,49.0240,0.9827,0.6242,0.2338,0.0648,...,0.01,0.17,0.16,-0.13,-0.13,-1.19,0.16,1.15,1.10,9
14300,1,199503.0,-0.6779,0.2836,29.9986,43.7132,0.8859,0.2278,0.0824,0.0468,...,1.32,-0.09,0.16,0.56,-0.33,1.00,1.07,-0.02,1.10,3
29579,1,199508.0,1.1883,0.2463,54.7405,63.0867,0.9999,0.9881,0.9742,0.5761,...,0.01,0.17,0.16,-0.13,-0.13,0.14,0.03,0.26,0.20,8
28217,1,199502.0,-2.1054,0.3637,17.2372,28.8934,0.0374,0.0174,0.0284,0.0371,...,1.32,-0.09,0.16,0.56,-0.33,1.00,1.07,-0.02,1.10,2
11561,2,199504.0,0.3538,0.2378,44.6988,54.4230,0.9979,0.8838,0.6065,0.1260,...,0.01,-0.71,-0.64,-0.24,0.22,-1.54,-0.60,-1.38,-1.11,4


- baseline modeling

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
pipe = Pipeline([('전처리',MinMaxScaler()), 
                 ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=300))])

In [ ]:
pipe.fit(x,y)

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, pipe.predict(val_x))

0.37103918228279387

In [ ]:
#0.38129
pipe_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : pipe.predict(test)})
pipe_base['X1STU30OCC2'] = pipe_base['X1STU30OCC2'].replace(idx2job)
pipe_base.to_csv('pipe_base.csv', index=False, encoding='utf-8-sig')

- one-vs-all modeling

In [ ]:
my_class = y.unique()
my_class

array([ 2, 11,  5,  0,  3,  6, 10,  8,  9, 13, 16,  1, 15,  4,  7, 12, 14,
       18, 17, 19, 20])

In [ ]:
my_models = []
for s in my_class:
    target = x.loc[y[y==s].index]
    #non_target = x.loc[y[y!=s].index]
    # undersampling코드
    non_target = x.loc[y[y!=s].index].sample(len(target))
    my_sample = pd.concat([target, non_target])
    my_sample = my_sample.sample(len(my_sample))
    my_y = pd.concat([pd.Series(1, index=y.loc[target.index].index),
                      pd.Series(0, y.loc[non_target.index].index)])
    my_y = my_y.loc[my_sample.index]

#### 모델 학습해서 my_models에 append, 1을 오바되게 잘 맞추면 점수 잘나옴
    pipe = Pipeline([('전처리',MinMaxScaler()), 
                     ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=300))])
    pipe.fit(my_sample, my_y)
####    
    my_models.append(pipe)

- Undersampling?
- Pipeline? 최적모델 탐색

In [ ]:
#21개 모델에 대한 예측 결과 생성(validation)
pred = np.zeros((len(val_x), len(my_class)))
for i, m in enumerate(my_models):
    p = m.predict_proba(val_x)[:,1]
    pred[:,i] = p
pred_class = [my_class[i] for i in pred.argmax(axis=1)]

In [ ]:
accuracy_score(val_y, pred_class)

0.36626916524701875

In [ ]:
#21개 모델에 대한 예측 결과 생성(test) : 0.37532
pred = np.zeros((len(test), len(my_class)))
for i, m in enumerate(my_models):
    p = m.predict_proba(test)[:,1]
    pred[:,i] = p
pred_class = [my_class[i] for i in pred.argmax(axis=1)]
one_vs_all = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : pred_class})
one_vs_all['X1STU30OCC2'] = one_vs_all['X1STU30OCC2'].replace(idx2job)
one_vs_all.to_csv('one_vs_all.csv', index=False, encoding='utf-8-sig')

In [ ]:
with open('data/one-vs-all.pkl', 'wb') as f:
    joblib.dump({'my_models':my_models},f)